In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


##### Step 1: Define Paths and Parameters

In [3]:
train_dir = os.path.join('Dataset', 'images.cv_jzk6llhf18tm3k0kyttxz', 'data', 'train')
test_dir = os.path.join('Dataset', 'images.cv_jzk6llhf18tm3k0kyttxz', 'data', 'test')
val_dir = os.path.join('Dataset', 'images.cv_jzk6llhf18tm3k0kyttxz', 'data', 'val')

In [4]:
# Define image size
img_height, img_width = 224, 224

# Define batch size
batch_size = 32

##### Step 2: Create Data Generators

In [5]:
# For training: apply data augmentation and rescaling.

train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0, 1]
    rotation_range=20,        # Random rotations up to 20 degrees
    zoom_range=0.2,           # Random zoom in/out
    horizontal_flip=True,     # Random horizontal flip
    width_shift_range=0.1,    # Random horizontal shifts
    height_shift_range=0.1    # Random vertical shifts
)

In [6]:
# For validation and test sets: only rescaling.

val_test_datagen = ImageDataGenerator(rescale=1./255)

##### Step 3: Load Images from Directories

In [7]:
# Training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 6225 images belonging to 11 classes.


In [8]:
# Validation generator
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1092 images belonging to 11 classes.


In [9]:
# Test generator
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3187 images belonging to 11 classes.


##### Step 4: Inspect the Loaded Data

In [10]:
print("Train Class Indices:", train_generator.class_indices)
print("Validation Class Indices:", val_generator.class_indices)
print("Test Class Indices:", test_generator.class_indices)

Train Class Indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food shrimp': 8, 'fish sea_food striped_red_mullet': 9, 'fish sea_food trout': 10}
Validation Class Indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food shrimp': 8, 'fish sea_food striped_red_mullet': 9, 'fish sea_food trout': 10}
Test Class Indices: {'animal fish': 0, 'animal fish bass': 1, 'fish sea_food black_sea_sprat': 2, 'fish sea_food gilt_head_bream': 3, 'fish sea_food hourse_mackerel': 4, 'fish sea_food red_mullet': 5, 'fish sea_food red_sea_bream': 6, 'fish sea_food sea_bass': 7, 'fish sea_food 

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
class_count = len(train_generator.class_indices)
class_count

11

##### Build a Deeper CNN Model

In [13]:
model = Sequential([
    # Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Block 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Block 4 - Additional convolutional block for deeper learning
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # Fully Connected Layers
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(class_count, activation='softmax')
])

c:\python_VSC\GUVI_Projects\guvi_projects_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','precision','recall','f1_score'])

In [15]:
# Print model summary to inspect the architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 26,874,411 (102.52 MB)

 Trainable params: 26,871,467 (102.51 MB)

 Non-trainable params: 2,944 (11.50 KB)

In [ ]:
# Define Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the Model with Early Stopping
epochs = 20

history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, validation_data=val_generator, validation_steps=val_generator.samples // batch_size, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 1335s 7s/step - accuracy: 0.4927 - f1_score: 0.4120 - loss: 1.5623 - precision: 0.6632 - recall: 0.3536 - val_accuracy: 0.1167 - val_f1_score: 0.0360 - val_loss: 12.3016 - val_precision: 0.1172 - val_recall: 0.1140
Epoch 2/10
  1/194 ━━━━━━━━━━━━━━━━━━━━ 22:07 7s/step - accuracy: 0.6250 - f1_score: 0.5526 - loss: 1.0571 - precision: 0.6667 - recall: 0.5000

c:\python_VSC\GUVI_Projects\guvi_projects_env\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


194/194 ━━━━━━━━━━━━━━━━━━━━ 49s 221ms/step - accuracy: 0.6250 - f1_score: 0.5526 - loss: 1.0571 - precision: 0.6667 - recall: 0.5000 - val_accuracy: 0.1204 - val_f1_score: 0.0380 - val_loss: 12.6567 - val_precision: 0.1227 - val_recall: 0.1149
Epoch 3/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 1242s 6s/step - accuracy: 0.6453 - f1_score: 0.5556 - loss: 1.0256 - precision: 0.7287 - recall: 0.5672 - val_accuracy: 0.2197 - val_f1_score: 0.0680 - val_loss: 7.8160 - val_precision: 0.2565 - val_recall: 0.2178
Epoch 4/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 50s 223ms/step - accuracy: 0.8438 - f1_score: 0.6171 - loss: 0.7562 - precision: 0.8889 - recall: 0.7500 - val_accuracy: 0.2188 - val_f1_score: 0.0674 - val_loss: 7.8591 - val_precision: 0.2489 - val_recall: 0.2178
Epoch 5/10
194/194 ━━━━━━━━━━━━━━━━━━━━ 1247s 6s/step - accuracy: 0.7727 - f1_score: 0.6872 - loss: 0.6511 - precision: 0.8264 - recall: 0.7220 - val_accuracy: 0.4816 - val_f1_score: 0.3595 - val_loss: 2.6769 - val_precision: 0.4990 - val_recall:

In [19]:
# Evaluate the Model on Test Data

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

99/99 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.8882 - f1_score: 0.4271 - loss: 0.3300 - precision: 0.8954 - recall: 0.8792


ValueError: too many values to unpack (expected 2)

In [ ]:
test_loss, test_accuracy, *other_metrics = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Visualize Training History

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()